In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import sklearn as skl
import tensorflow as tf

C:\Users\Sara\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Sara\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Sara\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Sara\anaconda3\envs\PythonData\lib\site-packages\ten

In [2]:
# Import our input dataset
charity_df = pd.read_csv('charity_data.csv')
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
#Generate our categorical variable list
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

#Check the number of unique values in each column
charity_df[charity_cat].nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
dtype: int64

In [4]:
#Drop the NAME column
charity_df = charity_df.drop(columns=["NAME", "EIN", "CLASSIFICATION"], axis = 1)
charity_df.head()

,APPLICATION_TYPE,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
#Rename APPLICATION column names
charity_df = charity_df.rename(columns ={"APPLICATION_TYPE": "application"})
charity_df.head()

,application,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,Heathcare,Trust,1,100000-499999,N,142590,1


In [6]:
#Generate our categorical variable list
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

#Check the number of unique values in each column
charity_df[charity_cat].nunique()

application               17
AFFILIATION                6
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [7]:
#check the unique value counts to see if binning is required
charity_df.application.value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: application, dtype: int64

In [8]:
#print the classification count
application_counts = charity_df.application.value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: application, dtype: int64

In [9]:
# Determine which values to replace
replace_application = list(application_counts[application_counts < 500].index)

# Replace in DataFrame
for application in replace_application:
    charity_df.application = charity_df.application.replace(application,"Other")


# Check to make sure binning was successful
charity_df.application.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: application, dtype: int64

In [10]:
charity_df[charity_cat].nunique()

application               9
AFFILIATION               6
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                9
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [11]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(charity_df[charity_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(charity_cat)
encode_df.head()

,application_Other,application_T10,application_T19,application_T3,application_T4,application_T5,application_T6,application_T7,application_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
# Merge one-hot encoded features and drop the originals
charity_df = charity_df.merge(encode_df,left_index=True, right_index=True)
charity_df = charity_df.drop(charity_cat,1)
charity_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,application_Other,application_T10,application_T19,application_T3,application_T4,application_T5,application_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
# Split our preprocessed data into our features and target arrays
y = charity_df["IS_SUCCESSFUL"].values
X = charity_df.drop(["IS_SUCCESSFUL"],1).values

# Use sklearn to split dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [14]:
# Create scaler instance
X_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
# Create the Keras Sequential model
nn_model = tf.keras.models.Sequential()

In [16]:
# Add our first Dense layer, including the input layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="tanh", input_dim=37))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [17]:
# Add the output layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [18]:
# Check the structure of the Sequential model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 38        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2         
Total params: 40
Trainable params: 40
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [20]:
# Fit the model to the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
25724/25724 [==============================] - 2s 61us/sample - loss: 0.6278 - acc: 0.6789
Epoch 2/150
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5836 - acc: 0.7239
Epoch 3/150
25724/25724 [==============================] - 1s 54us/sample - loss: 0.5809 - acc: 0.7254
Epoch 4/150
25724/25724 [==============================] - 1s 39us/sample - loss: 0.5804 - acc: 0.7264
Epoch 5/150
25724/25724 [==============================] - 1s 36us/sample - loss: 0.5803 - acc: 0.7257
Epoch 6/150
25724/25724 [==============================] - 1s 36us/sample - loss: 0.5801 - acc: 0.7257
Epoch 7/150
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5796 - acc: 0.7268
Epoch 8/150
25724/25724 [==============================] - 1s 36us/sample - loss: 0.5790 - acc: 0.7275
Epoch 9/150
25724/25724 [==============================] - 1s 36us/sample - loss: 0.5786 - acc: 0.7276
Epoch 10/150
25724/25724 [==============================] - 1s 36us/sampl

25724/25724 [==============================] - 1s 28us/sample - loss: 0.5766 - acc: 0.7282
Epoch 80/150
25724/25724 [==============================] - 1s 29us/sample - loss: 0.5766 - acc: 0.7284
Epoch 81/150
25724/25724 [==============================] - 1s 27us/sample - loss: 0.5766 - acc: 0.7284
Epoch 82/150
25724/25724 [==============================] - 1s 27us/sample - loss: 0.5765 - acc: 0.7281
Epoch 83/150
25724/25724 [==============================] - 1s 28us/sample - loss: 0.5765 - acc: 0.7278
Epoch 84/150
25724/25724 [==============================] - 1s 28us/sample - loss: 0.5765 - acc: 0.7276
Epoch 85/150
25724/25724 [==============================] - 1s 30us/sample - loss: 0.5766 - acc: 0.7289
Epoch 86/150
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5765 - acc: 0.7279
Epoch 87/150
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5765 - acc: 0.7277
Epoch 88/150
25724/25724 [==============================] - 1s 28us/sample - 

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.5816 - acc: 0.7208
Loss: 0.5816017411471108, Accuracy: 0.7208163142204285
